# Mixed Integer Linear Programming

## Planar Spacecraft with Two Thrusters

Based on the MIT notes for 16.410 Principles of Autonomy and Decision Making, ["Lecture 18: (Mixed-Integer) Linear Programming for Vehicle Routing and Motion Planning"](http://ocw.mit.edu/courses/aeronautics-and-astronautics/16-410-principles-of-autonomy-and-decision-making-fall-2010/lecture-notes/MIT16_410F10_lec18.pdf) by Emilio Frazolli, November 15, 2010.

Those notes describe a planar vehicle with four thrusters.

The basic structure of a linear dynamics model is $\dot{x} = A x + B u$

$$
\begin{bmatrix}
x_1(t +\Delta t)) \\ 
x_2(t +\Delta t) \\ 
\dot{x}_1(t +\Delta t) \\ 
\dot{x}_1(t +\Delta t)
\end{bmatrix} = 
\begin{bmatrix}
1 & 0 & \Delta t & 0 \\ 
0 & 1 & 0 & \Delta t \\ 
0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
x_1(t))\\ 
x_2(t)\\ 
\dot{x}_1(t)\\ 
\dot{x}_1(t)
\end{bmatrix} 
+
\begin{bmatrix}
\frac{1}{2}\Delta t^2 & 0 & -\frac{1}{2}\Delta t^2 & 0\\ 
0 & \frac{1}{2}\Delta t^2 & 0 & -\frac{1}{2}\Delta t^2 \\ 
\Delta t & 0 & \Delta t & 0\\ 
0 & \Delta t & 0 & \Delta t
\end{bmatrix}
\begin{bmatrix}
u_1^+(t))\\ 
u_2^+(t)\\ 
u_1^-(t))\\ 
u_2^-(t)
\end{bmatrix} 
$$

In [2]:
# This notebook has been built using conda, which ships with most important science packages in the Python ecosystem
# We need a Mixed Integer Linear Solver (GLPK is somewhat of an arbitrary choice). Unfortunately isn't a standard 
# part of cvxopt in conda. For the purposes of this notebook, it was built it separately, and I then pip installed
# cvxopt afterwards.
import cvxopt.glpk
import numpy as np

In [9]:
# implementation of the dynamics models
dt = 0.1
A = np.array([[1.0, 0.0, dt, 0.0],
              [0.0, 1.0, 0.0, dt],
              [0.0, 0.0, 1.0, 0.0],
              [0.0, 0.0, 0.0, 1.0]])

B = np.array([[0.5 * dt**2, 0.0, -0.5 * dt**2, 0.0],
              [0.0, 0.5 * dt**2, 0.0, -0.5 * dt**2],
              [dt, 0.0, dt, 0.0],
              [0.0, dt, 0.0, dt]])

In [10]:
print A
print B

[[ 1.   0.   0.1  0. ]
 [ 0.   1.   0.   0.1]
 [ 0.   0.   1.   0. ]
 [ 0.   0.   0.   1. ]]
[[ 0.005  0.    -0.005  0.   ]
 [ 0.     0.005  0.    -0.005]
 [ 0.1    0.     0.1    0.   ]
 [ 0.     0.1    0.     0.1  ]]


In [8]:
# initial state
x0 = np.matrix([[5.0, -3.0, 0.0, 0.0]]).T

In [11]:
print x0

[[ 5.]
 [-3.]
 [ 0.]
 [ 0.]]


In [14]:
# number of timesteps
n =40

# see Terminal constraints on slide 18
x = (n-1) - np.arange(n)
x = [ np.dot(np.linalg.matrix_power(A, i), B) for i in x]
x = np.hstack(x)

In [18]:
# glpk.lp reference 
#
# Solves a linear program using GLPK.
#
# (status, x, z, y) = lp(c, G, h, A, b)
# (status, x, z) = lp(c, G, h)
#
# PURPOSE
# (status, x, z, y) = lp(c, G, h, A, b) solves the pair of primal and
# dual LPs
#
#    minimize    c'*x            maximize    -h'*z + b'*y
#    subject to  G*x <= h        subject to  G'*z + A'*y + c = 0
#                A*x = b                     z >= 0.
#
#(status, x, z) = lp(c, G, h) solves the pair of primal and dual LPs
#
#    minimize    c'*x            maximize    -h'*z 
#    subject to  G*x <= h        subject to  G'*z + c = 0
#                                            z >= 0.

In [ ]:
# Constraints.
# Starting with a really simple set of constraints - every control input |u| < 1


In [15]:
G = np.vstack([np.identity(n), -np.identity(n)])
h = 2 * np.ones((2*n, 1))

_c = cvxopt.matrix(np.ones(n))
_G = cvxopt.matrix(G)
_h = cvxopt.matrix(h)
_A = cvxopt.matrix(x)
_b = cvxopt.matrix(-np.dot(A, x0))
s, u, z, y = cvxopt.glpk.lp(_c, _G, _h, _A, _b)


ValueError: incompatible dimensions

In [16]:
x = x0.copy()

for ui in u:
    x = np.dot(A, x) + np.dot(B, ui)
    #print x

NameError: name 'u' is not defined

In [69]:
for ui in u:
    print ui

2.0
2.0
2.0
2.0
2.0
2.0
2.0
1.52
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.52
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
